In [363]:
import numpy as np
import pandas as pd
import os
# for i in os.walk('C:\\Users\\baron\\OneDrive\\Documents\\cse6250group\\data'):
#     print(i)

In [364]:
#load data
isruc1 = pd.read_csv('C:\\Users\\baron\\OneDrive\\Documents\\cse6250group\\data\\isruc-sleep\\subgroupI\\scoring_i.csv')
isruc2 = pd.read_csv('C:\\Users\\baron\\OneDrive\\Documents\\cse6250group\\data\\isruc-sleep\\subgroupII\\scoring_ii.csv')
isruc3 = pd.read_csv('C:\\Users\\baron\\OneDrive\\Documents\\cse6250group\\data\\isruc-sleep\\subgroupIII\\scoring_iii.csv')
summary_isruc = pd.read_csv('C:\\Users\\baron\\OneDrive\\Documents\\cse6250group\\data\\isruc-sleep\\subgroupI\\scoring_summary_i.csv')

df = pd.read_csv('aggregated_sleep_data.csv')
cleaned_df = pd.read_csv('cleaned_sleep_stage_data.csv')

edf = pd.read_csv('C:\\Users\\baron\\OneDrive\\Documents\\cse6250group\\data\\sleep_edf_raw.csv')

C:\Users\baron\.conda\envs\BD4H\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [365]:
#cleaning
cleaned_df = cleaned_df[['epoch', 'nsrrid','sleep_stage']]
cleaned_df.columns = ['epoch', 'patient_id', 'stage']

isruc1 = isruc1[['epoch', 'stage', 'spo2', 'hr', 'events', 'bpos', 'txln','txex', 'technote', 'patient_id', 'score_id']]
isruc1['patient_id'] += 2000
isruc2 = isruc2[['epoch', 'stage', 'spo2', 'hr', 'events', 'bpos', 'txln','txex', 'technote', 'patient_id', 'score_id']]
isruc2['patient_id'] += 3000
isruc3 = isruc3[['epoch', 'stage', 'spo2', 'hr', 'events', 'bpos', 'txln','txex', 'technote', 'patient_id', 'score_id']]
isruc2['patient_id'] += 4000
isruc = pd.concat([isruc1, isruc2, isruc3])
isruc = isruc[['epoch', 'patient_id','stage']]
isruc['epoch'] = isruc['epoch'].astype('int')

edf[['epoch', 'patient_id', 'condition', 'EEG Fpz-Cz', 'EEG Pz-Oz','EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal','Event marker']]
edf = edf[['epoch', 'patient_id','condition']]
edf.columns = ['epoch', 'patient_id', 'stage']

In [366]:
#features
final_df = pd.concat([cleaned_df, isruc, edf])
final_df = final_df.replace('Sleep stage W', 'W')
final_df = final_df.replace('Sleep stage R', 'R')
final_df = final_df.replace('Sleep stage 1', '1')
final_df = final_df.replace('Sleep stage 2', '2')
final_df = final_df.replace('Sleep stage 3/4', '3')
final_df = final_df.replace('w', 'W')
final_df = final_df.merge(final_df.groupby(['patient_id','stage'], as_index = False).count(), left_on = ['patient_id','stage'], right_on = ['patient_id','stage'], how = 'left')
final_df = final_df.merge(final_df.groupby(['patient_id'], as_index = False).count(), left_on = ['patient_id'], right_on = ['patient_id'], how = 'left')
final_df = final_df[['patient_id', 'stage_x', 'epoch_y_x', 'epoch_y_y']]
final_df['%'] = final_df['epoch_y_x'] / final_df['epoch_y_y']
train_data = final_df[['patient_id', 'stage_x','%']].drop_duplicates().pivot(index = 'patient_id', columns = 'stage_x')
train_data.to_csv('train_data.csv')
train_data

,%,%,%,%,%,%,%,%,%,%,%,%,%,%,%,%
stage_x,0,1,2,3,5,1,2,3,N,N1,N2,N3,R,U,W,n2
patient_id,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,0.021099,0.090942,0.080029,NaN,0.083303,0.278283,0.133867,0.130593,0.000364,0.181521,NaN
2,NaN,NaN,NaN,NaN,NaN,0.036516,0.188275,0.035176,NaN,0.083752,0.216080,0.147404,0.172194,NaN,0.120603,NaN
3,NaN,NaN,NaN,NaN,NaN,0.035166,0.203891,0.035541,NaN,0.051627,0.191919,0.226337,0.143659,0.000374,0.111485,NaN
4,NaN,NaN,NaN,NaN,NaN,0.024016,0.190945,0.022441,NaN,0.106693,0.177953,0.124803,0.167717,NaN,0.185433,NaN
5,NaN,NaN,NaN,NaN,NaN,0.053154,0.198527,0.016971,NaN,0.057317,0.193724,0.108870,0.119757,0.000320,0.251361,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205654,0.157895,0.054737,0.604737,0.039474,0.143158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205655,0.060000,0.021290,0.490323,0.227097,0.201290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [367]:
summary_isruc=summary_isruc[['subject', 'age', 'sex', 'diagnosis', 'other problems']]
summary_isruc['subject'] += 2000
summary_isruc.columns = ['patient_id', 'age', 'sex', 'diagnosis', 'other problems']
df = df[['nsrrid', 'age_s1', 'gender']]
df = df.replace(1, 'M')
df = df.replace(2, 'F')
# edf= edf['patient_id'].drop_duplicates()
# edf.column = ['patient_id']
df.columns = ['patient_id','age', 'sex']
df = df.dropna()
test_data = pd.concat([summary_isruc, df])
test_data.to_csv('test_data.csv')
test_data

,patient_id,age,sex,diagnosis,other problems
0,2001,64.0,M,SAOS,Depression
1,2002,52.0,M,SAOS,Restless leg syndrome
2,2003,38.0,M,REM Sleep Behaviour Disorder,PLMS
3,2004,27.0,M,SRVAS,Epilepsy
4,2005,58.0,F,SAOS,Insomnia
...,...,...,...,...,...
481,205654,67.0,M,NaN,NaN
482,205655,58.0,F,NaN,NaN
483,205657,63.0,M,NaN,NaN
484,205668,68.0,F,NaN,NaN


In [368]:
data = train_data.merge(test_data, on ='patient_id')

from sklearn.model_selection import train_test_split

X = data
y = data['diagnosis']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

C:\Users\baron\.conda\envs\BD4H\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [369]:
data

,patient_id,"(%, 0)","(%, 1)","(%, 2)","(%, 3)","(%, 5)","(%, 1)","(%, 2)","(%, 3)","(%, N)",...,"(%, N2)","(%, N3)","(%, R)","(%, U)","(%, W)","(%, n2)",age,sex,diagnosis,other problems
0,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.230682,0.262500,0.136932,NaN,0.304545,NaN,64.0,M,SAOS,Depression
1,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.400415,0.136929,0.105809,NaN,0.237033,0.000519,52.0,M,SAOS,Restless leg syndrome
2,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.326617,0.170201,0.259809,0.001060,0.132556,NaN,38.0,M,REM Sleep Behaviour Disorder,PLMS
3,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001038,...,0.439252,0.239356,0.242471,0.001038,0.026999,NaN,27.0,M,SRVAS,Epilepsy
4,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.329714,0.188000,0.049714,NaN,0.347429,NaN,58.0,F,SAOS,Insomnia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,205654,0.157895,0.054737,0.604737,0.039474,0.143158,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,67.0,M,NaN,NaN
582,205655,0.060000,0.021290,0.490323,0.227097,0.201290,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,58.0,F,NaN,NaN
583,205657,0.257927,0.057927,0.465854,0.103049,0.115244,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,63.0,M,NaN,NaN
584,205668,0.077340,0.065517,0.529064,0.114778,0.213300,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,F,NaN,NaN
